# `swift_too` module

## Swift_ObsQuery example - querying past *Swift* observations

### API version = 1.0

#### Author: Jamie A. Kennea (Penn State)

The Swift_ObsQuery class allows for querying the database of observations have have already been performed by Swift, otherwise known as the "As-Flown Science Timeline" (AFST). Note this will only fetch observations that have already been performed, not scheduled observations. 

In [1]:
%matplotlib notebook
import matplotlib.pyplot as plt
from swifttools.swift_too import Swift_ObsQuery
from time import sleep
import numpy as np

### Constructing the query

Set up the class, giving our username and shared secret. These can be set up [on the Swift TOO website](https://www.swift.psu.edu/toop/too.php). After you log in, you will find your shared secret, and can modify it as necessary under the [Update Account Info](https://www.swift.psu.edu/toop/account_info.php) link. 

In [2]:
query = Swift_ObsQuery()
query.username = 'myuser'
query.shared_secret = 'mysharedsecret'

First example, how often has Swift observed the binary system SS 433? Well, I can't remember the RA/Dec off the top of my head, so let's look it up on Simbad.

In [3]:
target_name = "SS 433"
from astroquery.simbad import Simbad
table = Simbad.query_object(target_name)

Convert this Simbad obtained RA/Dec into an Astropy SkyCoord

In [4]:
from astropy.coordinates import SkyCoord
import astropy.units as u

query.skycoord = SkyCoord(f"{table['RA'][0]} {table['DEC'][0]}",frame="icrs",unit=(u.hourangle, u.deg))


RA/Dec is stored by the TOO API in decimal degrees, in J2000 epoch as this is the epoch Swift uses

In [5]:
print(f"RA/Dec(J2000) = {query.ra:.4f}, {query.dec:.4f}")

RA/Dec(J2000) = 287.9565, 4.9827


Looks legit. However, we can also set RA/Dec the old fashioned way which doesn't require Simbad or astropy

In [6]:
query.ra, query.dec = 287.9565, 4.9827

Note that if you already defined a skycoord, it will remain correct even if you changed the ra and dec properties.

In [7]:
query.skycoord

<SkyCoord (FK5: equinox=J2000.000): (ra, dec) in deg
    (287.9565, 4.9827)>


Swift_ObsQuery has a default search radius which is....

In [8]:
print(f"Default search radius = {query.radius:.3f} degrees")

Default search radius = 0.197 degrees


That's 12 arcminutes, which is the approximate field of view of Swift's X-ray Telescope (XRT). We can narrow that down a bit, so we only get matches that are in the center of the field of view (FOV), and also in UVOT which has a smaller FOV.

In [9]:
query.radius = 5 / 60 # 5 arc-minutes, as the units for radius are degrees

### Submitting the query 

OK let's query the Swift timeline to see how many observations it's taken. This might take a few seconds to process. Note that there are two ways you can submit a job. You can either queue it up, then check periodically if the job is complete, then if it is, continue. Here is some example code of doing that.

In [10]:
if query.queue():
    while(not query.complete):
        print(f"Waiting for job #{query.status.jobnumber} to be processed ...")
        sleep(2)
    print(f"Done. Job #{query.status.jobnumber} complete")
else:
    print(f"Request rejected. Error: {query.status.errors}")

Waiting for job #3063 to be processed ...
Done. Job #3063 complete


Or you can let the TOO_API code handle the timeout. Note that the default timeout is set to 120, but you can set it longer if you like, using e.g. query.submit(timeout=300). Note: You can still run the code below if you ran the previous entry, if the query is completed, it will just return back as True.


In [11]:
if query.submit():
    print("Success!")
else:
    print(f"Fail or timeout? {query.status}")

Success!


### Examining the results of the query

So how many observations has Swift taken of this target?

In [12]:
print(f"This many: {len(query)}")

This many: 77


That's a lot of damage. Here's a thing to remember, every entry in this is a single snapshot of observation. As Swift is in a low Earth orbit, it means that a snapshot is typically max 30 mins, or sometimes a bit longer (~44 mins), so a long exposure will consist of multiple snapshots. Observations are grouped by obsid (a 12 digit number with leading zeros), so snapshots with the same obsid are part of the same planned observation.

In [13]:
_ = [print(observation) for observation in query]

2005-07-30 13:23:02 - 2005-07-30 13:27:00 Target: SS433           (00035190001) Exp:    95s Slewtime:   143s
2005-07-30 13:27:02 - 2005-07-30 13:30:48 Target: SS433           (00035190002) Exp:   210s Slewtime:    16s
2005-08-01 13:36:02 - 2005-08-01 13:40:00 Target: SS433           (00035190005) Exp:    95s Slewtime:   143s
2005-08-01 13:40:02 - 2005-08-01 13:56:58 Target: SS433           (00035190006) Exp:  1000s Slewtime:    16s
2005-08-01 18:44:15 - 2005-08-01 18:46:32 Target: SS433           (00035190007) Exp:     0s Slewtime:   137s
2005-08-01 18:46:35 - 2005-08-01 19:08:01 Target: SS433           (00035190008) Exp:  1270s Slewtime:    16s
2005-08-18 00:59:02 - 2005-08-18 01:14:58 Target: SS433           (00035190013) Exp:   870s Slewtime:    86s
2005-08-18 02:35:02 - 2005-08-18 02:51:58 Target: SS433           (00035190013) Exp:   930s Slewtime:    86s
2006-06-30 13:45:02 - 2006-06-30 13:55:00 Target: SS 433          (00035190014) Exp:   425s Slewtime:   173s
2006-06-30 15:19:02

Wow that is a lot of observations. 

### Pointing Accuracy

Here's an interesting thing about Swift, it doesn't point very accurately. This is because the ACS system sacrifices accuracy for speed. The goal is to get the object of interest into the field of view of XRT and UVOT, not at the boresight. As a result, the pointing direction be typically up to 3 arcminutes off the requested pointing direction. Note that for each entry listed above, we give an ra and dec value, so let's check out the variation:

In [14]:
plt.figure()
plt.plot([entry.ra for entry in query],[entry.dec for entry in query],"+")
plt.plot([entry.ra_point for entry in query],[entry.dec_point for entry in query],"X")
plt.xlabel("RA(J2000)")
plt.ylabel("Dec(J2000)")
_= plt.title(f"{target_name} pointing scatter")


<IPython.core.display.Javascript object>

As you can see there's a lot of variation of the pointing direction. Each entry also has a values ra_point, dec_point, which give the decimal degrees values of the requested pointing direction for each observation. This typically will be the coordinates of the Target of the Observation, but sometimes if offsets are applied for any reason, it might differ.

The ra_point/dec_point values aren't necessarily going to be for the object you queried on. In fact there can be multiple values of ra_point/dec_point if the queried field lies inside multiple pointings.

Although the RA/Dec are returned in J2000 decimal degrees, it's pretty easy to convert these into SkyCoords for you astropy fans. As the swift_too module doesn't require astropy as a depency, you can do it with some simple codeas follows:

In [15]:
sc = SkyCoord([entry.ra for entry in query],[entry.dec for entry in query],frame='fk5',unit=(u.deg,u.deg))

However, sometimes ra_point, dec_point cannot be determined, so the value will be 'None'. I'm going to filter those out so we can do some comparing.

In [16]:
sc = SkyCoord([entry.ra for entry in query if entry.ra_point != None],[entry.dec for entry in query if entry.ra_point != None],frame='fk5',unit=(u.deg,u.deg))
scp = SkyCoord([entry.ra_point for entry in query  if entry.ra_point != None],[entry.dec_point for entry in query  if entry.ra_point != None],frame='fk5',unit=(u.deg,u.deg))

I made an array of ra_point/dec_point so we can evaluate how accurately Swift actually pointed at this target. Let's make a histogram of the pointing offsets.

In [17]:
plt.figure()
plt.hist(sc.separation(scp).arcmin,bins=30)
plt.ylabel("Number of pointings")
plt.xlabel("Offset from requested pointing direction (arc-minutes)")
print(f"Median offset value = {np.median(sc.separation(scp).arcmin):0.2f} arc-minutes")

<IPython.core.display.Javascript object>

Median offset value = 2.17 arc-minutes


So you'll see that the median offset here is around 2 arc-minutes, with a pretty big scatter, and there may even be some large outliers. 


### Grouping Snapshots into Observations 

Let's take a look at an individual entry to see what information is being returned by this query.

In [18]:
query[0].api_data

{'begin': '2005-07-30 13:23:02',
 'settle': '2005-07-30 13:25:25',
 'end': '2005-07-30 13:27:00',
 'ra': 287.994,
 'dec': 4.972,
 'roll': 317.225,
 'targname': 'SS433',
 'targetid': 35190,
 'seg': 1,
 'ra_point': 287.9565,
 'dec_point': 4.982667,
 'xrt': 'Auto',
 'uvot': '0x000a'}

So some useful information here. Firstly remember each entry represents a snapshot of Swift data, that is data taken in a single orbit of observations. Typically data that you obtain from the SDC will be grouped by observation, and those observations can contain many snapshots. Observations have a unique target ID (target ID) and segment (seg) numbers. These typically are combined into a Observation ID (obsnum), which in the SDC format will look like a concatibation of the target ID, and segment, with padding zeros, e.g.:

In [19]:
print(f"Target ID = {query[0].targetid}, segment = {query[0].seg}, ObservationID = {query[0].obsnum}")

Target ID = 35190, segment = 1, ObservationID = 00035190001


If you're interested in all the observations under a particular Observation ID, then there's a property called "observations" that contains a dictionary of all observations on an Observation ID basis. Let's look at the summary of this dictionary by just printing out all the entries.

In [20]:
_ = [print(obs) for obs in query.observations.values()]

2005-07-30 13:23:02 - 2005-07-30 13:27:00 Target: SS433           (00035190001) Exp:    95s Slewtime:   143s
2005-07-30 13:27:02 - 2005-07-30 13:30:48 Target: SS433           (00035190002) Exp:   210s Slewtime:    16s
2005-08-01 13:36:02 - 2005-08-01 13:40:00 Target: SS433           (00035190005) Exp:    95s Slewtime:   143s
2005-08-01 13:40:02 - 2005-08-01 13:56:58 Target: SS433           (00035190006) Exp:  1000s Slewtime:    16s
2005-08-01 18:44:15 - 2005-08-01 18:46:32 Target: SS433           (00035190007) Exp:     0s Slewtime:   137s
2005-08-01 18:46:35 - 2005-08-01 19:08:01 Target: SS433           (00035190008) Exp:  1270s Slewtime:    16s
2005-08-18 00:59:02 - 2005-08-18 02:51:58 Target: SS433           (00035190013) Exp:  1800s Slewtime:   172s
2006-06-30 13:45:02 - 2006-06-30 23:27:00 Target: SS 433          (00035190014) Exp:  3275s Slewtime:  1212s
2006-07-01 02:34:02 - 2006-07-02 10:53:56 Target: SS 433          (00035190015) Exp:  7830s Slewtime:  2600s
2006-11-03 18:42:58

You can see that now the summary shows the details for an entire observation, with the start and end times being those of associated with the first and last observation of that Observation ID, and the exposure time being the total. Importantly due to orbit gaps, the exposure time is not just end minus begin. Each entry in the observations dictionary contains details on the individual segments also. Note that Observation ID is a string, given as it is formatted with padding zeros. For example:

In [21]:
_ = [print(snapshot) for snapshot in query.observations['00035190015']]

2006-07-01 02:34:02 - 2006-07-01 02:43:56 Target: SS 433          (00035190015) Exp:   435s Slewtime:   159s
2006-07-01 04:11:02 - 2006-07-01 04:24:56 Target: SS 433          (00035190015) Exp:   675s Slewtime:   159s
2006-07-01 05:47:02 - 2006-07-01 05:59:01 Target: SS 433          (00035190015) Exp:   560s Slewtime:   159s
2006-07-01 07:24:02 - 2006-07-01 07:34:58 Target: SS 433          (00035190015) Exp:   485s Slewtime:   171s
2006-07-01 09:00:02 - 2006-07-01 09:11:57 Target: SS 433          (00035190015) Exp:   535s Slewtime:   180s
2006-07-01 10:36:02 - 2006-07-01 10:47:57 Target: SS 433          (00035190015) Exp:   535s Slewtime:   180s
2006-07-01 12:13:02 - 2006-07-01 12:23:58 Target: SS 433          (00035190015) Exp:   485s Slewtime:   171s
2006-07-01 20:15:02 - 2006-07-01 20:25:58 Target: SS 433          (00035190015) Exp:   475s Slewtime:   181s
2006-07-01 21:51:02 - 2006-07-01 22:02:57 Target: SS 433          (00035190015) Exp:   535s Slewtime:   180s
2006-07-02 02:41:02

You can query the exposure and other infomation for the combined snapshots in the Observation ID.

In [22]:
query.observations['00035190015'].exposure

datetime.timedelta(seconds=7830)

Note that the result here is a datetime timedelta object. You can easily get the seconds as an integer or convert to an astropy TimeDelta object.

In [23]:
query.observations['00035190015'].exposure.seconds

7830

In [24]:
from astropy.time import TimeDelta
TimeDelta(query.observations['00035190015'].exposure)

<TimeDelta object: scale='None' format='datetime' value=2:10:30>

Note that there is no RA/Dec (ra/dec) for an observation, only the requested pointing direction (ra_point/dec_point), because actual RA/Dec will be different for each snapshot, so delve into the individual snapshots for those. 

In [25]:
query.observations['00035190015'].ra_point, query.observations['00035190015'].dec_point

(287.9565, 4.982667)

# Instrument Configuration 

All observations for a given Observation ID will have the same instrument configuration. Let's check those out.

In [26]:
print(f"XRT mode = {query.observations['00035190015'].xrt}, UVOT mode = {query.observations['00035190015'].uvot}")

XRT mode = Auto, UVOT mode = 0x20ed


In this case the XRT mode is `Auto`, which means that XRT itself decides whether to be in PC or WT mode, based on the brightness of sources in the central 200x200 pixels of the detector, roughly the central 8.5 arcmin x 8.5 arcmin box. Because of this we can't determine what mode XRT will have actually taken the data in without looking at it. However for many observation, the mode is fixed, here you will see results like so:

In [27]:
print(f"XRT mode = {query.observations['00035190037'].xrt}")

XRT mode = PC


As this is PC mode, we can guarantee that the data are taken in PC mode.

For UVOT the mode above is a hex number `0x20ed`. There are a large number of modes that can be used with UVOT, given it's many different combinations of filters, exposure windows, etc. You can look at the [UVOT mode table](https://www.swift.psu.edu/operations/mode_lookup.php) to help choose a UVOT mode. You can also look up what UVOT modes contain using the following link: https://www.swift.psu.edu/operations/mode_table.php?hexval=0x20ed

Here's what that looks like:

In [28]:
from IPython.core.display import HTML
import requests
HTML(requests.get(f"https://www.swift.psu.edu/operations/mode_table.php?hexval={query.observations['00035190015'].uvot}").text)

Filter,Event FOV,Image FOV,Bin Size,Max. Exp. Time,Weighting,Comments
W1,,17,2,540,1500,BLUE WEIGHTING
UU,,17,2,270,750,BLUE WEIGHTING
BB,,17,2,270,750,BLUE WEIGHTING
W2,17,,,,3000,BLUE WEIGHTING
VV,,17,2,270,750,BLUE WEIGHTING
M2,17,,,,2250,BLUE WEIGHTING


It's also important to remember that a specific UVOT mode does not guarantee data was taken with the expected filters, so always check the data.